In [1]:
%load_ext rpy2.ipython

In [2]:
%%R
source("scripts/TADA/TADAaddEvidence.R")
source("scripts/TADA/likelihoodFunctionCC.R")

#################################################################
# Application of TADA
#################################################################

# Model parameters: two categories of mutations - LoF and mis3 mutations ("probably damaging" by PolyPhen2)
mu.frac <- c(0.074, 0.32)
gamma.mean.dn <- c(20, 4.7)
beta.dn <- c(1,1)
gamma.mean.CC <- c(2.3, 1.00)
beta.CC <- c(4.0, 1000)
rho1 <- c(0.1, 0.5)
nu1 <- c(200, 100)
rho0 <- c(0.1, 0.5)
nu0 <- c(200, 100)
hyperpar <- as.array(rbind(gamma.mean.dn, beta.dn, gamma.mean.CC, beta.CC, rho1, nu1, rho0, nu0))
l <- 100
pi0 <- 0.94 # the fraction of non-risk genes

# ASC (Autism Sequencing Consortium) data
# The file name contains the sample size information
# The only relevant counts are dn.LoF and dn.mis3
data <- read.csv("scripts/TADA/data/ASC_2231trios_1333trans_1601cases_5397controls.csv", header=TRUE, as.is=TRUE)
ntrio <- 2231  # number of trios
ncase <- 1601  # number of cases
nctrl <- 5397  # number of controls
ntrans <- 1333 # number of subjects with transmission data
N <- list(dn=ntrio, ca=ntrans+ncase, cn=ntrans+nctrl)


counts <- as.array(cbind(data$dn.LoF, data$case.LoF+data$trans.LoF, data$ctrl.LoF+data$ntrans.LoF, data$dn.mis3, data$case.mis3+data$trans.mis3, data$ctrl.mis3+data$ntrans.mis3))

dim(counts)
colnames(counts) <- c("dn", "case.LoF", "ctrl.LoF", 1:3)
head(counts)


     dn case.LoF ctrl.LoF 1  2  3
[1,]  0        0        1 0  7  7
[2,]  0        2        3 0  3 14
[3,]  0        0        0 0  0  0
[4,]  0        2        5 0 10 28
[5,]  0        5       22 0 11 40
[6,]  0        1        2 0  3  6


In [4]:
%%R
#Only de novo data
piSeq <- seq(0.01, 0.12, by = 0.01)
gamma.dn.LoF.seq <- seq(10, 30, by = 0.25)
beta.dn.LoF.seq <- seq(0.5, 2.5, by = 0.25)
nG <- length(gamma.dn.LoF.seq)
nPi <- length(piSeq)
nBeta <- length(beta.dn.LoF.seq)

matrixLLK <- matrix(0, ncol = 4, nrow = nBeta*nG*nPi)
indexI <- 1

dim(matrixLLK)

[1] 8748    4


In [5]:
%%R
indexI <- 1
for (ii in 1:nG){
    
    for (jj in 1:nPi){
        for (kk in 1:nBeta){
            
        matrixLLK[indexI, 2] <- piSeq[jj]

         matrixLLK[indexI, 1] <- gamma.dn.LoF.seq[ii]
   
            matrixLLK[indexI, 3] <- beta.dn.LoF.seq[kk]
        indexI <- indexI + 1
                }}
}

In [6]:
%%R
head(matrixLLK)

     [,1] [,2] [,3] [,4]
[1,]   10 0.01 0.50    0
[2,]   10 0.01 0.75    0
[3,]   10 0.01 1.00    0
[4,]   10 0.01 1.25    0
[5,]   10 0.01 1.50    0
[6,]   10 0.01 1.75    0


In [7]:
%%R
library(data.table)
matrixLLK0 <- data.table(matrixLLK)

//anaconda3/lib/python3.4/site-packages/rpy2/robjects/functions.py:106: UserWarning: data.table 1.9.6  For help type ?data.table or https://github.com/Rdatatable/data.table/wiki

  res = super(Function, self).__call__(*new_args, **new_kwargs)
//anaconda3/lib/python3.4/site-packages/rpy2/robjects/functions.py:106: UserWarning: The fastest way to learn (by data.table authors): https://www.datacamp.com/courses/data-analysis-the-data-table-way

  res = super(Function, self).__call__(*new_args, **new_kwargs)


In [8]:
%%R
head(matrixLLK0)

   V1   V2   V3 V4
1: 10 0.01 0.50  0
2: 10 0.01 0.75  0
3: 10 0.01 1.00  0
4: 10 0.01 1.25  0
5: 10 0.01 1.50  0
6: 10 0.01 1.75  0


In [10]:
%%R
hyperTemp <- hyperpar
system.time(llk0 <- apply(matrixLLK0, 1, function(x){
    
     hyperTemp[1:2, 1] <- x[1:2]
    
        outLLK <- marginal(hyperpar = hyperTemp, pi = x[3], 
                           counts = counts, N = N, mu = data$mut.rate*mu.frac[1],
                           denovo.only = TRUE)$marginal
    return(outLLK)
            
}))

//anaconda3/lib/python3.4/site-packages/rpy2/robjects/functions.py:106: UserWarning: Error in dnbinom(x, gamma.mean * beta, beta/(beta + 2 * N * mu)) : 
  Interrupted

  res = super(Function, self).__call__(*new_args, **new_kwargs)
//anaconda3/lib/python3.4/site-packages/rpy2/robjects/functions.py:106: UserWarning: In addition: 
  res = super(Function, self).__call__(*new_args, **new_kwargs)
//anaconda3/lib/python3.4/site-packages/rpy2/robjects/functions.py:106: UserWarning: There were 50 or more warnings (use warnings() to see the first 50)
  res = super(Function, self).__call__(*new_args, **new_kwargs)
//anaconda3/lib/python3.4/site-packages/rpy2/robjects/functions.py:106: UserWarning: 

  res = super(Function, self).__call__(*new_args, **new_kwargs)


KeyboardInterrupt: 

In [11]:
%%R
hyperTemp <- hyperpar

   testLLK <- function(x, y, z){
       hyperTemp[1, 1] <- x
       hyperTemp[2, 1] <- y
       
       marginal(hyperpar = hyperTemp, pi = z, 
                           counts = counts, N = N, mu = data$mut.rate*mu.frac[1],
                           denovo.only = TRUE)$marginal
   }

Timing stopped at: 676.444 1.26 675.088 


In [13]:
%%R
head(matrixLLK0)

   V1   V2   V3 V4
1: 10 0.01 0.50  0
2: 10 0.01 0.75  0
3: 10 0.01 1.00  0
4: 10 0.01 1.25  0
5: 10 0.01 1.50  0
6: 10 0.01 1.75  0


In [14]:
%%R
system.time(xyz <- testLLK(x = matrixLLK[, 1], y = matrixLLK[, 2], z = matrixLLK[, 3]))

Timing stopped at: 0 0 0.001
Error in hyperTemp[1, 1] <- x : 
  number of items to replace is not a multiple of replacement length


//anaconda3/lib/python3.4/site-packages/rpy2/robjects/functions.py:106: UserWarning: Error in hyperTemp[1, 1] <- x : 
  number of items to replace is not a multiple of replacement length

  res = super(Function, self).__call__(*new_args, **new_kwargs)
